In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/season/2023/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Completions', 'Passing Attempts', 'Completion Percentage', 'Passing Yards',
                 'Avg Yards per Pass Attempt', 'Passing Yards per Game', 'Longest Pass', 'Passing Touchdowns', 'Interceptions',
                 'Total Sacks', 'Sack Yards Lost', 'Adjusted QBR', 'Passer Rating']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,QB,17,388,560,69.3,"4,624",8.3,272.0,78,29,14,29,171,61.5,101.1
1,QB,17,407,605,67.3,"4,575",7.6,269.1,70,30,12,30,197,61.1,97.9
2,QB,17,410,590,69.5,"4,516",7.7,265.6,92,36,9,39,255,73.4,105.9
3,QB,17,385,579,66.5,"4,306",7.4,253.3,81,29,18,24,152,70.3,92.2
4,QB,16,308,444,69.4,"4,280",9.6,267.5,76,31,11,28,153,73.4,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,WR,16,0,0,0.0,0,0.0,0.0,0,0,0,1,8,0.0,39.6
111,RB,11,0,0,0.0,0,0.0,0.0,0,0,0,1,6,0.1,39.6
112,WR,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,3.3,39.6
113,WR,12,1,2,50.0,-1,-0.5,-0.1,0,0,0,0,0,3.2,56.3


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Tua Tagovailoa,MIA,QB,17,388,560,69.3,"4,624",8.3,272.0,78,29,14,29,171,61.5,101.1
1,2,Jared Goff,DET,QB,17,407,605,67.3,"4,575",7.6,269.1,70,30,12,30,197,61.1,97.9
2,3,Dak Prescott,DAL,QB,17,410,590,69.5,"4,516",7.7,265.6,92,36,9,39,255,73.4,105.9
3,4,Josh Allen,BUF,QB,17,385,579,66.5,"4,306",7.4,253.3,81,29,18,24,152,70.3,92.2
4,5,Brock Purdy,SF,QB,16,308,444,69.4,"4,280",9.6,267.5,76,31,11,28,153,73.4,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,92,Amon-Ra St. Brown,DET,WR,16,0,0,0.0,0,0.0,0.0,0,0,0,1,8,0.0,39.6
111,92,De'Von Achane,MIA,RB,11,0,0,0.0,0,0.0,0.0,0,0,0,1,6,0.1,39.6
112,92,Garrett Wilson,NYJ,WR,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,3.3,39.6
113,114,Christian Kirk,JAX,WR,12,1,2,50.0,-1,-0.5,-0.1,0,0,0,0,0,3.2,56.3


In [7]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Tua Tagovailoa,MIA,QB,17,388,560,69.3,"4,624",8.3,272.0,78,29,14,29,171,61.5,101.1
1,2,Jared Goff,DET,QB,17,407,605,67.3,"4,575",7.6,269.1,70,30,12,30,197,61.1,97.9
2,3,Dak Prescott,DAL,QB,17,410,590,69.5,"4,516",7.7,265.6,92,36,9,39,255,73.4,105.9
3,4,Josh Allen,BUF,QB,17,385,579,66.5,"4,306",7.4,253.3,81,29,18,24,152,70.3,92.2
4,5,Brock Purdy,SF,QB,16,308,444,69.4,"4,280",9.6,267.5,76,31,11,28,153,73.4,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,92,Amon-Ra St. Brown,DET,WR,16,0,0,0.0,0,0.0,0.0,0,0,0,1,8,0.0,39.6
111,92,De'Von Achane,MIA,RB,11,0,0,0.0,0,0.0,0.0,0,0,0,1,6,0.1,39.6
112,92,Garrett Wilson,NYJ,WR,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,3.3,39.6
113,114,Christian Kirk,JAX,WR,12,1,2,50.0,-1,-0.5,-0.1,0,0,0,0,0,3.2,56.3


In [9]:
total_stats.to_csv("Passing_Stats_2023.csv", index = False)
print("Data saved to 2023_Passing_Stats.csv")

Data saved to 2023_Passing_Stats.csv
